# Idea

## Motivation
In the Laji dataset, the abundance of each species (i.e. the number of individuals) is assumed to be the number of observations. However, this assumption will not hold in the case that the observations recorded are influenced by other factors such as the number of gatherings in one location. Therefore, the goal of using statistic model is to re-evaluate species distribution with consideration for such uncertainty, as well as creating inferences for species distribution in areas with low or no observations using observations in their surrounding areas.

## Model


### Beta - Bernoulli model
Considering one species in an area:  
Observations can be classified as $1$ for individuals that belong to that species, or $0$ for individuals that do not.  
Assume that such observations $X = \{x_{1}, x_{2}, ...,x_{n}\}$ are Bernoulli distributed with a beta prior:

$$x \sim Bern(\theta), \theta \sim Beta(\alpha, \beta), \theta \in [0, 1]$$

Currently, the **uninformative uniform prior** is used: $Beta(1,1)$

### Dirichlet prior
Dirichlet is the multivariate version of Beta distribution, now our models can be described as:

$$x \sim multi(M, \theta), ∑_{k=1}^{K}{\theta_{k}} = 1$$
$$\theta \sim Dir(\alpha), \alpha_{1}, \alpha_{2}, ..., \alpha_{k} > 0$$

Currently, the **uninformative uniform prior** is used: $Dir(1,..,1)$

### Neighboring observations
Observations of the surrounding areas are also used to update the posterior model. However, these observations are "penalized" by a certain term.  
Currently, a penalty of 0.5 is used.

# Implementation

## Requirements

In [ ]:
!pip install plotly==5.15.0
!pip install scikit-fda==0.8.1
!pip install scipy==1.11.2
!pip install h3==3.7.6
!pip install geojson==3.0.1
!pip install shapely==2.0.1

## Data preprocessing

In [ ]:
import h3
import pandas as pd
import numpy as np
import json

In [ ]:
df = pd.read_csv('tree_data_sample.csv')
df.dropna(subset=['lat', 'lon'], inplace=True)
df.head()

,date,lat,lon,municipality,taxonId,scientificName,verbatim
0,2023-09-13,60.29276,24.56223,Espoo,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris
1,2023-09-11,62.90000,23.30000,"Kuortane, Lapua, Seinäjoki",http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris
2,2023-09-10,61.34076,22.13714,Harjavalta,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris
3,2023-09-09,60.18495,24.93093,Helsinki,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris
4,2023-09-09,60.18457,24.93013,Helsinki,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris


In [ ]:
class h3_grid():
  def __init__(self, resolution=5):
    self.resolution = resolution
    self.grid = False

  def row_to_h3cell(self, row) -> str:
    """
    Function to return the h3 grid index of the input row.
    :param pd.Series row: The row to be loaded.
    """
    return h3.geo_to_h3(lat=row['lat'], lng=row['lon'], resolution = self.resolution)

  def fit(self, df: pd.DataFrame):
    """
    Function to load data into a hexagonal grid.
    :param pd.DataFrame df: The data frame to be loaded.
    """
    df['h3_cell'] = df.apply(self.row_to_h3cell, axis=1)
    df = df.reset_index(drop=False).groupby('h3_cell').index.agg(list).to_frame("observations_id").reset_index()
    df['count'] = df['observations_id'].apply(lambda row: len(row))
    df['neighbors'] = df['h3_cell'].apply(lambda h3_index: h3.k_ring(h3_index, 1))
    self.grid = df

  def grid_info(self) -> pd.DataFrame:
    """
    Function to return the h3 grid groupings as a data frame.
    :param pd.DataFrame df: The data frame to be loaded.
    :return pd.DataFrame: A data frame with columns for cell index, list of cells in the grid, a count of observations in the cell, and the indices of the neigbours of the cell.
    """
    if not isinstance(self.grid, pd.DataFrame):
      raise Exception('Data not loaded. Use fit() first.')

    return self.grid

  def find_neighbors (self, cell_id : str) -> list:
    neighbors = self.grid.loc[self.grid['h3_cell'] == cell_id, 'neighbors']
    return list(eval(str(neighbors.tolist()[0])))

In [ ]:
grid_object = h3_grid()
grid_object.fit(df)
grid_object.grid_info().head()

,h3_cell,observations_id,count,neighbors
0,85012613fffffff,"[9562, 78059, 109502, 113234]",4,"{8501268bfffffff, 85012613fffffff, 85012617fff..."
1,8501261bfffffff,"[473, 9335, 9345, 9426, 9427, 9485, 9510, 9536...",27,"{85012657fffffff, 850126cffffffff, 85012613fff..."
2,85012643fffffff,"[76368, 107634]",2,"{85012647fffffff, 85012657fffffff, 85012643fff..."
3,85012647fffffff,"[76362, 76369, 107631, 109126]",4,"{85012673fffffff, 85012647fffffff, 85012657fff..."
4,8501264bfffffff,"[102955, 107707, 109417]",3,"{85013587fffffff, 85012643fffffff, 8501264ffff..."


In [ ]:
grid_object.find_neighbors("85012613fffffff")

['8501268bfffffff',
 '8501261bfffffff',
 '8501268ffffffff',
 '85012613fffffff',
 '850126c7fffffff',
 '85012617fffffff',
 '85012603fffffff']

In [ ]:
df.head()

,date,lat,lon,municipality,taxonId,scientificName,verbatim,h3_cell
0,2023-09-13,60.29276,24.56223,Espoo,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris,85089963fffffff
1,2023-09-11,62.90000,23.30000,"Kuortane, Lapua, Seinäjoki",http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris,85088967fffffff
2,2023-09-10,61.34076,22.13714,Harjavalta,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris,85088a97fffffff
3,2023-09-09,60.18495,24.93093,Helsinki,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris,851126d3fffffff
4,2023-09-09,60.18457,24.93013,Helsinki,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris,851126d3fffffff


In [ ]:
def group_observations (dataframe, columns):
    grouped_df = dataframe.groupby(columns).count()
    grouped_df.reset_index(inplace = True)
    columns_to_drop = list(filter(lambda name : name not in columns, grouped_df.columns))
    grouped_df = grouped_df.drop(columns = columns_to_drop[1:])
    grouped_df.columns = columns + ["nof_obs"]
    return grouped_df

# scientificName or taxonId works
grouped_df = group_observations(df, ['h3_cell', 'scientificName'])

def pivot_obs_dataframe (dataframe, species_identifier, area_id):
    pivoted_df = dataframe.pivot(index = area_id, columns = species_identifier, values = "nof_obs")
    pivoted_df = pivoted_df.fillna(0.0)
    return pivoted_df

pivoted_df = pivot_obs_dataframe(grouped_df, "scientificName", "h3_cell")
pivoted_df

scientificName,Ginkgo biloba,Pinus banksiana,Pinus contorta,Pinus contorta var. latifolia,Pinus koraiensis,Pinus mugo,Pinus mugo subsp.mugo,Pinus peuce,Pinus sibirica,Pinus strobus,Pinus sylvestris,Pinus sylvestris var. lapponica,Pinus sylvestris var. sylvestris
h3_cell,,,,,,,,,,,,,
85012613fffffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
8501261bfffffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0.0,0.0
85012643fffffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
85012647fffffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
8501264bfffffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
852a33dbfffffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
852aa38ffffffff,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
852ec31bfffffff,0.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def aggregate_neighbouring_rows (grid, dataframe, cell_id):
    neighbors = grid.find_neighbors (cell_id)
    neighbors_df = dataframe[dataframe.index.isin(neighbors)]
    neighbors_sum = neighbors_df.sum().to_dict()
    neighbors_sum['h3_cell'] = cell_id
    return neighbors_sum

agg_neighbors_df = pd.DataFrame(
    list(pivoted_df.apply(lambda row : aggregate_neighbouring_rows(grid_object, pivoted_df, row.name), axis=1)
))
agg_neighbors_df.set_index('h3_cell', inplace=True)
agg_neighbors_df

,Ginkgo biloba,Pinus banksiana,Pinus contorta,Pinus contorta var. latifolia,Pinus koraiensis,Pinus mugo,Pinus mugo subsp.mugo,Pinus peuce,Pinus sibirica,Pinus strobus,Pinus sylvestris,Pinus sylvestris var. lapponica,Pinus sylvestris var. sylvestris
h3_cell,,,,,,,,,,,,,
85012613fffffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,135.0,0.0,0.0
8501261bfffffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,167.0,1.0,0.0
85012643fffffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0
85012647fffffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0
8501264bfffffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
852a33dbfffffff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
852aa38ffffffff,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
852ec31bfffffff,0.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Beta distribution of single species

In [ ]:
from scipy.stats import beta

def update_prior_with_neighbor (area_row, species_identifier, penalty = 0.5):
    successes = area_row[species_identifier]
    trials = area_row[:-1].sum()
    alpha = 1. + successes*penalty
    beta = 1. + (trials - successes)*penalty
    return alpha, beta

def update_posterior_with_actual (prior_alpha, prior_beta, area_row, species_identifier):
    successes = area_row[species_identifier]
    trials = area_row[:-1].sum()
    alpha = prior_alpha + successes
    beta = prior_beta + (trials - successes)
    return alpha, beta

def calculate_dist_params (area_id, species_identifier, df, agg_neighbors_df, penalty = 0.5):
    prior_alpha, prior_beta = update_prior_with_neighbor(agg_neighbors_df.loc[area_id], species_identifier, penalty)
    alpha, beta = update_posterior_with_actual(prior_alpha, prior_beta, df.loc[area_id], species_identifier)
    return {
        "h3_cell" : area_id,
        "prior_alpha" : prior_alpha,
        "prior_beta" : prior_beta,
        "alpha" : alpha,
        "beta" :  beta
    }

def generate_dist_beta_df (df, agg_neighbors_df, species_identifier):
    dist_df = pd.DataFrame(list(df.apply(lambda row : calculate_dist_params(
        row.name, species_identifier, df, agg_neighbors_df), axis = 1)))
    dist_df[["mean", "variance", "skew"]] = dist_df.apply(lambda row : beta.stats(row["alpha"], row["beta"], moments = "mvs"), axis=1, result_type='expand')
    dist_df["median"] = dist_df.apply(lambda row : beta.median(row["alpha"], row["beta"]), axis=1)
    dist_df["50%"] = dist_df.apply(lambda row : beta.interval(0.5, row["alpha"], row["beta"]), axis=1)
    dist_df["90%"] = dist_df.apply(lambda row : beta.interval(0.9, row["alpha"], row["beta"]), axis=1)
    dist_df["95%"] = dist_df.apply(lambda row : beta.interval(0.95, row["alpha"], row["beta"]), axis=1)
    return dist_df

generate_dist_beta_df (pivoted_df, agg_neighbors_df, "Pinus sylvestris")

,h3_cell,prior_alpha,prior_beta,alpha,beta,mean,variance,skew,median,50%,90%,95%
0,85012613fffffff,68.5,1.0,72.5,1.0,0.986395,0.000180,-1.919987,0.990485,"(0.9810603509005991, 0.9960398336372733)","(0.9595216066307184, 0.9992927564989766)","(0.9503916692624902, 0.9996508498225039)"
1,8501261bfffffff,84.5,1.5,111.5,1.5,0.986726,0.000115,-1.579408,0.989470,"(0.9817859222333355, 0.9945894587301612)","(0.965638634364104, 0.998426975019801)","(0.9590349027698565, 0.9990349347829742)"
2,85012643fffffff,17.0,1.0,19.0,1.0,0.950000,0.002262,-1.720334,0.964176,"(0.9296353550084063, 0.9849728893336261)","(0.8541314966877566, 0.9973039936971287)","(0.8235330881930346, 0.9986683711958205)"
3,85012647fffffff,5.5,1.0,9.5,1.0,0.904762,0.007493,-1.496325,0.929635,"(0.8642218932816056, 0.9701715927222317)","(0.7295406136340671, 0.9946152558442426)","(0.6782067473487566, 0.9973385156269132)"
4,8501264bfffffff,13.0,1.0,16.0,1.0,0.941176,0.003076,-1.674727,0.957603,"(0.9170040432046712, 0.9821805485552589)","(0.8292502770175191, 0.9967993022898115)","(0.7940927857921773, 0.9984188882772341)"
...,...,...,...,...,...,...,...,...,...,...,...,...
2094,852a33dbfffffff,1.0,4.5,1.0,9.5,0.095238,0.007493,1.496325,0.070365,"(0.029828407277768303, 0.13577810671839444)","(0.005384744155757441, 0.27045938636593286)","(0.0026614843730867277, 0.32179325265124337)"
2095,852aa38ffffffff,1.0,2.0,1.0,4.0,0.200000,0.026667,1.049781,0.159104,"(0.0693951408979004, 0.2928932188134525)","(0.012741455098566189, 0.527129195498412)","(0.0063094632097098705, 0.6023646356164746)"
2096,852ec31bfffffff,1.0,29.0,1.0,85.0,0.011628,0.000132,1.931420,0.008122,"(0.003378774022912073, 0.01617706814575628)","(0.0006032684825592491, 0.034630074970687034)","(0.0002978122096584484, 0.04247033991124916)"
2097,85314293fffffff,1.0,2.0,1.0,4.0,0.200000,0.026667,1.049781,0.159104,"(0.0693951408979004, 0.2928932188134525)","(0.012741455098566189, 0.527129195498412)","(0.0063094632097098705, 0.6023646356164746)"


## Dirichlet distribution of all species

In [ ]:
from scipy.stats import dirichlet

def update_prior_alphas (area_row, penalty = 0.5):
    successes = area_row.to_numpy()
    return 1. + successes*penalty

def update_posterior_alphas (prior_alphas, area_row):
    successes = area_row.to_numpy()
    return prior_alphas + successes

def calculate_dist_alphas (area_id, df, agg_neighbors_df, penalty = 0.5):
    prior_alphas = update_prior_alphas (agg_neighbors_df.loc[area_id], penalty = penalty)
    alphas = update_posterior_alphas (prior_alphas, df.loc[area_id])
    return alphas

def generate_dist_dirichlet_df (df, agg_neighbors_df):
    alphas_df = pd.DataFrame(columns = df.columns)
    for row in df.apply(lambda row : calculate_dist_alphas(row.name, df, agg_neighbors_df), axis = 1):
        alphas_df.loc[len(alphas_df.index)] = list(row)
    alphas_df['h3_cell'] = df.index
    alphas_df.set_index('h3_cell', inplace = True)

    means_df = pd.DataFrame(columns = df.columns)
    for row in alphas_df.apply(lambda row : dirichlet.mean(row.to_numpy()), axis = 1):
        means_df.loc[len(means_df.index)] = list(row)
    means_df['h3_cell'] = df.index
    means_df.set_index('h3_cell', inplace = True)

    vars_df = pd.DataFrame(columns = df.columns)
    for row in alphas_df.apply(lambda row : dirichlet.var(row.to_numpy()), axis = 1):
        vars_df.loc[len(vars_df.index)] = list(row)
    vars_df['h3_cell'] = df.index
    vars_df.set_index('h3_cell', inplace = True)
    return alphas_df, means_df, vars_df

alphas_df, means_df, vars_df = generate_dist_dirichlet_df (pivoted_df, agg_neighbors_df)
alphas_df

scientificName,Ginkgo biloba,Pinus banksiana,Pinus contorta,Pinus contorta var. latifolia,Pinus koraiensis,Pinus mugo,Pinus mugo subsp.mugo,Pinus peuce,Pinus sibirica,Pinus strobus,Pinus sylvestris,Pinus sylvestris var. lapponica,Pinus sylvestris var. sylvestris
h3_cell,,,,,,,,,,,,,
85012613fffffff,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,72.5,1.0,1.0
8501261bfffffff,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,111.5,1.5,1.0
85012643fffffff,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,19.0,1.0,1.0
85012647fffffff,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,9.5,1.0,1.0
8501264bfffffff,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,16.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
852a33dbfffffff,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,9.5,1.0,1.0,1.0
852aa38ffffffff,2.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.5,1.0,1.0,1.0
852ec31bfffffff,1.0,1.0,1.0,1.0,85.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
means_df

scientificName,Ginkgo biloba,Pinus banksiana,Pinus contorta,Pinus contorta var. latifolia,Pinus koraiensis,Pinus mugo,Pinus mugo subsp.mugo,Pinus peuce,Pinus sibirica,Pinus strobus,Pinus sylvestris,Pinus sylvestris var. lapponica,Pinus sylvestris var. sylvestris
h3_cell,,,,,,,,,,,,,
85012613fffffff,0.011834,0.011834,0.011834,0.011834,0.011834,0.011834,0.011834,0.011834,0.011834,0.011834,0.857988,0.011834,0.011834
8501261bfffffff,0.008065,0.008065,0.008065,0.008065,0.008065,0.008065,0.008065,0.008065,0.008065,0.008065,0.899194,0.012097,0.008065
85012643fffffff,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,0.612903,0.032258,0.032258
85012647fffffff,0.046512,0.046512,0.046512,0.046512,0.046512,0.046512,0.046512,0.046512,0.046512,0.046512,0.441860,0.046512,0.046512
8501264bfffffff,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.571429,0.035714,0.035714
...,...,...,...,...,...,...,...,...,...,...,...,...,...
852a33dbfffffff,0.046512,0.046512,0.046512,0.046512,0.046512,0.046512,0.046512,0.046512,0.046512,0.441860,0.046512,0.046512,0.046512
852aa38ffffffff,0.156250,0.062500,0.062500,0.062500,0.062500,0.062500,0.062500,0.062500,0.062500,0.156250,0.062500,0.062500,0.062500
852ec31bfffffff,0.010309,0.010309,0.010309,0.010309,0.876289,0.010309,0.010309,0.010309,0.010309,0.010309,0.010309,0.010309,0.010309


In [ ]:
vars_df

scientificName,Ginkgo biloba,Pinus banksiana,Pinus contorta,Pinus contorta var. latifolia,Pinus koraiensis,Pinus mugo,Pinus mugo subsp.mugo,Pinus peuce,Pinus sibirica,Pinus strobus,Pinus sylvestris,Pinus sylvestris var. lapponica,Pinus sylvestris var. sylvestris
h3_cell,,,,,,,,,,,,,
85012613fffffff,0.000137,0.000137,0.000137,0.000137,0.000137,0.000137,0.000137,0.000137,0.000137,0.000137,0.001425,0.000137,0.000137
8501261bfffffff,0.000064,0.000064,0.000064,0.000064,0.000064,0.000064,0.000064,0.000064,0.000064,0.000064,0.000725,0.000096,0.000064
85012643fffffff,0.000976,0.000976,0.000976,0.000976,0.000976,0.000976,0.000976,0.000976,0.000976,0.000976,0.007414,0.000976,0.000976
85012647fffffff,0.001971,0.001971,0.001971,0.001971,0.001971,0.001971,0.001971,0.001971,0.001971,0.001971,0.010961,0.001971,0.001971
8501264bfffffff,0.001188,0.001188,0.001188,0.001188,0.001188,0.001188,0.001188,0.001188,0.001188,0.001188,0.008445,0.001188,0.001188
...,...,...,...,...,...,...,...,...,...,...,...,...,...
852a33dbfffffff,0.001971,0.001971,0.001971,0.001971,0.001971,0.001971,0.001971,0.001971,0.001971,0.010961,0.001971,0.001971,0.001971
852aa38ffffffff,0.007755,0.003447,0.003447,0.003447,0.003447,0.003447,0.003447,0.003447,0.003447,0.007755,0.003447,0.003447,0.003447
852ec31bfffffff,0.000104,0.000104,0.000104,0.000104,0.001106,0.000104,0.000104,0.000104,0.000104,0.000104,0.000104,0.000104,0.000104


## Visualization

# Conclusion

## Model comparison analysis
- **Beta distribution** of a single species works best if studying one dominant species because it assumes **uniform prior distribution** of equal probabilities of one individual belonging to one species or not.
- **Dirichlet distribution** of all species gives a better overall view about the distribution of species, as the distribution of each species is considered, and the individual expected value of the percentage of each species is calculated. However, as it assumes **uniform prior distribution** of equal probabilities of one individual belonging to each of the different species, the posterior distribution might not be suitable for areas with fewer species present.
- Regardless of the areas, **Dirichlet distribution** always considers more uncertainty than **Beta distribution** as its **uniform prior distribution** gives weight to more categories.

## Future directions

### Species pruning for Dirichlet distribution
One strategy to minimize the impact of **uniform prior distribution** of **Dirichlet distribution** in areas with fewer species is to prune the list of species to be considered in the **Dirichlet distribution**. In particular, the prior distribution will be formulated for 2 groups of species:
- For species that have recorded occurrences in one area or its surrounding areas, the prior alpha is $1$.
- For species without recorded occurrences in one area or its surrounding areas, the prior alpha is $1/n$ with $n$ being the total number of species without recorded occurences.  

This approach expands on the **uniform prior distribution** of the **beta distribution** which gives equal probabilities for one individual to belong to one species or not. Essentially, this approach groups all species without recorded occurrences in one area as the "unrecorded species" and gives it the prior alpha as $1$ (i.e. equal probability with other species). Such probability is then equally divided between members of this group, giving an individual prior alpha of $1/n$ (n being the total number of species in this group).

This approach regularizes the impact of the **uninformative uniform prior** on the **Dirichlet distribution** on areas with fewer species, while still considering the uncertainty of unrecorded species.


### Testing on different penalty terms